In [2]:
import os
import json
import datetime

#use whichever folder
folder = #use whichever folder the files are stored in

#quick function to open files
def open_file(file_name):
    with open(os.path.join(folder, file_name), mode = 'r', encoding = 'utf8') as reader:
        dct = json.load(reader)
    return dct

In [45]:
#Open Weekend Lineup
weekend_lineup_dict = open_file('Weekend Artist Lineup.json')

In [46]:
#Get set info for artist of interest
def get_artist_set_info_dict(artist_name):
    return open_file('Weekend Artist Lineup.json')[artist_name]

get_artist_set_info_dict('Ariana Grande')

{'Day': 'Sunday, April 21, 2019',
 'Start Time': 'Sunday, April 21, 2019 10:30 PM -0700',
 'End Time': 'Monday, April 22, 2019 12:00 AM -0700',
 'Stage': ' Coachella Stage'}

In [7]:
#tweet timestamp to time converter
def get_tweet_timestamp(tid):
    offset = 1288834974657
    tstamp = (tid >> 22) + offset
    utcdttime = datetime.datetime.fromtimestamp(tstamp/1000, tz=datetime.timezone.utc)
    return utcdttime

In [37]:
#Get list of tweets from artist set
def get_tweets_from_set(artist_name):
    artist_set_info_dict = get_artist_set_info_dict(artist_name)
    hour_tweet_list = []
    artist_tweet_list = []
    
    start_time = datetime.datetime.strptime(artist_set_info_dict['Start Time'], '%A, %B %d, %Y %I:%M %p %z')
    start_day = start_time.utctimetuple().tm_mday
    start_hour = start_time.utctimetuple().tm_hour
    
    
    end_time = datetime.datetime.strptime(artist_set_info_dict['End Time'], '%A, %B %d, %Y %I:%M %p %z')
    end_day = end_time.utctimetuple().tm_mday
    end_hour = end_time.utctimetuple().tm_hour
    
    first_hour = f'{start_day}.{start_hour}'
    last_hour = f'{end_day}.{end_hour}'
    
    if start_day == end_day:
        day_dct = open_file('4.' + str(start_day) + '.19 coachella tweets.json')
        if end_hour - start_hour == 0:
            hour_tweet_list = [day_dct[first_hour]]
        elif end_hour - start_hour == 1:
            hour_tweet_list = [day_dct[first_hour], day_dct[last_hour]]
        else:
            hour_tweet_list = [day_dct[first_hour], day_dct[f'{start_day}.{start_hour + 1}'], day_dct[last_hour]]
    else:
        day_start_dct = open_file('4.' + str(start_day) + '.19 coachella tweets.json')
        day_end_dct = open_file('4.' + str(end_day) + '.19 coachella tweets.json')
        if end_hour - start_hour == -23:
            hour_tweet_list = [day_start_dct[first_hour], day_end_dct[last_hour]] # for when tweet hour ranges from hour 23 to hour 0
        else:
            if start_hour == 23:
                hour_tweet_list = [day_start_dct[first_hour], day_end_dct[f'{end_day}.{end_hour-1}'], day_end_dct[last_hour]] #for when tweet hour ranges from hour 23 - hour 1
            else:
                hour_tweet_list = [day_start_dct[first_hour], day_start_dct[f'{start_day}.{start_hour+1}'], day_end_dct[last_hour]] #for when tweet hour ranges from hour 22 - hour 0
        
    hour_tweet_list = [element for row in hour_tweet_list for element in row]
    hour_tweet_list = sorted(hour_tweet_list)
    
    for tweet_id in hour_tweet_list:
        if start_time < get_tweet_timestamp(int(tweet_id)) < end_time:
            artist_tweet_list.append(tweet_id)
    
        
    return artist_tweet_list
      

In [48]:
ariana_grande = get_tweets_from_set('Ariana Grande')

In [49]:
get_tweet_timestamp(int(ariana_grande[0])), get_tweet_timestamp(int(ariana_grande[-1]))

(datetime.datetime(2019, 4, 22, 5, 30, 0, 236000, tzinfo=datetime.timezone.utc),
 datetime.datetime(2019, 4, 22, 6, 59, 59, 993000, tzinfo=datetime.timezone.utc))

In [72]:
len(ariana_grande)

29073

Now that you have the list of tweet_ids you can pull the full tweet objects using Twython

In [42]:
from twython import Twython
from time import sleep
import itertools

#consumer_key = add consumer key here
#consumer_secret = add consumer secret here
#access_token = add access token here
#access_token_secret = add access token secret here

#twitter = Twython(consumer_key, consumer_secret, access_token, access_token_secret)

In [73]:
#using the lookup_status function (with tweet_mode in 'exteneded' for any tweets longer than 140 characters) to access tweets

tweets = twitter.lookup_status(id = ariana_grande[10:20], tweet_mode='extended')
for status in tweets:
    print(status['created_at'], status['full_text'], sep = '\n')
    print()

Mon Apr 22 05:30:03 +0000 2019
eh baby isn't Ariana supposed to be performing now?

Mon Apr 22 05:30:02 +0000 2019
@prayingforYG @ygofficialblink Thấy bảo BP chỉ là leadgroup cho main group của lão thôi nên lão mới để vậy. Đ ai ngờ nhóm bật như tên lửa vậy

Mon Apr 22 05:30:02 +0000 2019
RT @prasejeebus: Is everyone okay https://t.co/upcSSo11M1

Mon Apr 22 05:30:03 +0000 2019
Chaelice
#BLACKPINK 
#LISA #Rosé #chaelisa https://t.co/C5xFFNcKtH

Mon Apr 22 05:30:02 +0000 2019
RT @forevernews_in: Sansa changed her mind instantly for brienne. tormund again was in awe of her. jaime knighted her. a room full of men a…

Mon Apr 22 05:30:02 +0000 2019
uhm i thought it's going to be ariana's performance........

Mon Apr 22 05:30:03 +0000 2019
RT @guitarrmar: I luv Coachella https://t.co/X1lZyt30Mf

Mon Apr 22 05:30:03 +0000 2019
@Park_Choy @BTS_twt Lol girl same

Mon Apr 22 05:30:03 +0000 2019
are the live streams late again tonight like they were last weekend?? cause I can’t find Ari’s stream y

In [32]:
#twitter allows you to request up to 100 unique tweet ids in a lookup_status request.
#function to group by 100 

def grouper(iterable, n=100):
    it = iter(iterable)
    while True:
        chunk = list(itertools.islice(it, n))
        if not chunk:
            return
        yield chunk

In [75]:
#creating the tweet dictionary.  Using a 1 second sleep timer to not max the rate limit 
tweets_dict = {}
for tweets in grouper(ariana_grande, 100):
    rslts = twitter.lookup_status(id=','.join(tweets), tweet_mode='extended')
    for tweet in rslts:
        tweets_dict[tweet['id']] = tweet
    sleep(1)

In [91]:
#tweets_dict.keys()

In [90]:
#example tweet
tweets_dict[1120198304170532864]['id'], tweets_dict[1120198304170532864]['full_text']

(1120198304170532864,
 "Let's watch Ariana Grande &amp; Perfume #Coachella 2nd weekend")